In [1]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import torch
import random
from scipy.sparse import csgraph
import sys
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.optim import lr_scheduler
import copy
from utils import *
from graphConvolution import *
from cluster import *
from torch.backends import cudnn
import torch.nn as nn
import torch.nn.functional as F
cudnn.benchmark = False            # if benchmark=True, deterministic will be False
cudnn.deterministic = True

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid,bias=True)
        self.gc2 = GraphConvolution(nhid, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return x
    
best_acc2 = 0
test_acc2 = 0
def train2(epoch, model,record):
    global best_acc2
    global test_acc2
    global model_best2
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features_GCN, adj)
    loss_train = F.cross_entropy(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    model.eval()
    output = model(features_GCN, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    record[acc_val.item()] = acc_test.item()
    
    if acc_val > best_acc2:
        print("###########################update testacc#####################################")
        print("current val_acc:", acc_val.item())
        print("current test_acc:", acc_test.item())
        best_acc2 = acc_val
        model_best2 = copy.deepcopy(model)
        test_acc2 = acc_test.item()

In [3]:
class LR(nn.Module):
    def __init__(self, nfeat, nclass, dropout):
        super(LR, self).__init__()
        self.lr1 = nn.Linear(nfeat, nclass)
        
        self.dropout = dropout

    def forward(self, x):
        x = F.dropout(x, self.dropout, training=self.training)
        
        x = self.lr(x)
        return x

class MLP(nn.Module):
    def __init__(self, nfeat,nhid, nclass, dropout):
        super(MLP, self).__init__()
        self.lr1 = nn.Linear(nfeat, nhid)
        self.lr2 = nn.Linear(nhid, nclass)
        
        self.dropout = dropout

    def forward(self, x):
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.lr1(x))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.lr2(x)
        return x

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def aug_normalized_adjacency(adj):
   adj = adj + sp.eye(adj.shape[0])
   adj = sp.coo_matrix(adj)
   row_sum = np.array(adj.sum(1))
   d_inv_sqrt = np.power(row_sum, -0.5).flatten()
   d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
   d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
   return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()


def aug_random_walk(adj):
   adj = adj + sp.eye(adj.shape[0])
   adj = sp.coo_matrix(adj)
   row_sum = np.array(adj.sum(1))
   d_inv = np.power(row_sum, -1.0).flatten()
   d_mat = sp.diags(d_inv)
   return (d_mat.dot(adj)).tocoo()

def train(epoch, model,record):
    global best_acc
    global model_best
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features)
    loss_train = F.cross_entropy(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    model.eval()
    output = model(features)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])

    
    if acc_val > best_acc:
        print("############################################################################")
        print("current val_acc:", acc_val.item())
        print("current test_acc:", acc_test.item())
        best_acc = acc_val
        model_best = copy.deepcopy(model)
        
def get_normalized_similarity(dist):
    if len(dist) == 0:
        return {}
    elif len(dist) == 1:
        return {list(dist.keys())[0]:1}
    else:
        index_value = dist
        simi_values = []
        for index,value in index_value.items():
            simi_values.append(1-value)
        normalized_values = (simi_values - min(simi_values))/(max(simi_values)-min(simi_values))
        count = 0
        for index,value in index_value.items():
            index_value[index] = normalized_values[count]
            count+=1
        return index_value

def get_normalized_difference(dist):
    if len(dist) == 0:
        return {}
    elif len(dist) == 1:
        return {list(dist.keys())[0]:1}
    else:
        index_value = dist
        diff_values = []
        for index,value in index_value.items():
            diff_values.append(value)
        normalized_values = (diff_values - min(diff_values))/(max(diff_values)-min(diff_values))
        count = 0
        for index,value in index_value.items():
            index_value[index] = normalized_values[count]
            count+=1
        return index_value


def distEclud(vecA, vecB):
    return np.sqrt(sum(np.power(vecA - vecB, 2)))

def average_ab(num_selected,all_left):
    target_sum=sum(num_selected)

    while(True):
        for i in range(len(num_selected)):
            num_selected[i]=min(num_selected[i],all_left[i])
        sum_num=sum(num_selected)
        if(sum_num==target_sum):
            break
        cnt=0
        for i in range(len(num_selected)):
            if(num_selected[i]<all_left[i]):
                cnt+=1
        a_tmp=(target_sum-sum_num)//cnt
        b_tmp=(target_sum-sum_num)%cnt
        cnt_tmp=0
        for i in range(len(num_selected)):
            if(num_selected[i]<all_left[i]):
                if(cnt_tmp<b_tmp):
                    num_selected[i]+=a_tmp+1
                    cnt_tmp+=1
                else:
                    num_selected[i]+=a_tmp

def get_weighted_dist(output1,output2,alpha1,alpha2):
    dist12 = output1 - output2
    weighted_dist = []
    for node in range(num_node):
        weighted_dist.append(np.sqrt(sum(np.power(dist12[node], 2))))
    weighted_dist = np.array(weighted_dist)
    weighted_dist = weighted_dist*alpha1*alpha2
    return weighted_dist

def get_normalized_difference(dist):
    if len(dist) == 0:
        return {}
    elif len(dist) == 1:
        return {list(dist.keys())[0]:1}
    else:
        index_value = dist
        diff_values = []
        for index,value in index_value.items():
            diff_values.append(value)
        normalized_values = (diff_values - min(diff_values))/(max(diff_values)-min(diff_values))
        count = 0
        for index,value in index_value.items():
            index_value[index] = normalized_values[count]
            count+=1
        return index_value

def cut_zero(last_node):
    cnt_zero=0
    for i in range(last_node.shape[0]):
        if(last_node[i]<0):
            cnt_zero+=-last_node[i]
            last_node[i]=0
    i=0
    while(True):
        #print(last_node, cnt_zero)
        if(last_node[i]>0 and cnt_zero>0):
            last_node[i]-=1
            cnt_zero-=1
        if(cnt_zero==0):
            break
        i=(i+1)%num_class

def get_receptive_fields_dense(cur_neighbors, selected_node, weighted_score): 
    receptive_vector=((cur_neighbors+adj_matrix[selected_node])!=0)+0
    count=weighted_score.dot(receptive_vector)
    return count

def get_current_neighbors_dense(cur_nodes):
    neighbors=(adj_matrix[list(cur_nodes)].sum(axis=0)!=0)+0
    return neighbors

def get_max_receptive_node_dense(idx_used,high_score_nodes,weighted_score): 
    t=time.time()
    max_receptive_field = 0
    max_receptive_node = 0
    _weighted_score=np.array(list(weighted_score.values()))
    cur_neighbors=get_current_neighbors_dense(idx_used)
    for node in high_score_nodes:
        receptive_field=get_receptive_fields_dense(cur_neighbors,node,_weighted_score)
        if receptive_field > max_receptive_field:
            max_receptive_field = receptive_field
            max_receptive_node = node
        
    return max_receptive_node

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

### 1.Data processing

In [4]:
set_seed(4)
T = 19 
num_node = 3327 
num_class = 6 
initial_num_per_class = 2
ititial_labeled_nodes = num_class*initial_num_per_class
num_for_label = num_class*20 
total_training_epochs = 1500 

proxy_dp = 0.6
proxy_lr = 0.2
hidden_size = 128
m_T = 0.3
m_1 = 1
m = np.zeros(T)
for t in range(T):
    m_t = (m_T-m_1)*(t)/(T-1) +1
    m[t] = m_t
m = m/np.sum(m)
batch_sizes = np.ones(T)*2
batch_sizes = np.array([2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
print('batch_sizes_per_class:',batch_sizes )
print('batch_sizes:',batch_sizes*num_class )

alphas = [] 
for temp_T in range(T+1):
    alpha = np.cos(np.pi*temp_T/(2*(T)))
    alphas.append(alpha)
print('alphas:',alphas)

batch_epochs = (m*total_training_epochs).astype(int) 
print('batch_epochs:',batch_epochs)

batch_sizes_per_class: [2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
batch_sizes: [12  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6]
alphas: [1.0, 0.9965844930066698, 0.9863613034027223, 0.9694002659393304, 0.9458172417006346, 0.9157733266550574, 0.8794737512064891, 0.8371664782625287, 0.7891405093963936, 0.7357239106731317, 0.6772815716257411, 0.6142127126896679, 0.5469481581224269, 0.47594739303707345, 0.40169542465296953, 0.32469946920468357, 0.24548548714079924, 0.16459459028073375, 0.0825793454723324, 6.123233995736766e-17]
batch_epochs: [121 116 112 107 102  97  93  88  83  78  74  69  64  60  55  50  45  41
  36]


In [5]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(dataset="citeseer")


[STEP 1]: Upload citeseer dataset.
| # of nodes : 3327
| # of edges : 4614.0
| # of features : 3703
| # of clases   : 6
| # of train set : 120
| # of val set   : 500
| # of test set  : 1000


/home/zwt/SIGMOD2021_ALG/examples/utils.py:30: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [6]:
labels_npy = labels.cpu().numpy()
initial_node_index = [[] for i in range(num_class)]
for i in range(num_for_label):
    node_class = labels_npy[i]
    initial_node_index[node_class].append(i)
    
for i in range(num_class):
    initial_node_index[i] = random.sample(initial_node_index[i],initial_num_per_class)
initial_node_index = np.array(initial_node_index).ravel()

nodes_index = []
for i in range(num_node): 
    if i not in initial_node_index:
        nodes_index.append(i)
nodes_index = list(initial_node_index)+nodes_index

idx_avaliable = []
for i in range(num_for_label): 
    if i not in initial_node_index:
        idx_avaliable.append(i)
        
for i in range(num_for_label,num_node):
    if i not in idx_val and i not in idx_test:
        idx_avaliable.append(i)

idx_train = torch.LongTensor(nodes_index[:len(initial_node_index)])
idx_unlabeled = nodes_index[len(initial_node_index):] 

In [7]:
adj = aug_normalized_adjacency(adj)
adj_tensor = torch.FloatTensor(adj.todense())
adj_tensor = adj_tensor.cuda()

adj_matrix = np.array(torch.mm(adj_tensor,adj_tensor).cpu()) 
adj_tensor = torch.mm(adj_tensor,adj_tensor) 

adj = sparse_mx_to_torch_sparse_tensor(adj).float() 
adj = adj.cuda()

In [8]:
features_GCN = copy.deepcopy(features)
features = features.cuda()
features = np.array(torch.mm(adj_tensor, features).cpu()) 

train_label = np.array(labels[idx_train]).reshape(-1,1)
train_embeddings = np.array(features[idx_train])

labeled_data = np.hstack((train_embeddings,train_label))
unlabeled_data =  np.array(features[idx_unlabeled])

clusterAssment,centroids = semi_kMeans(labeled_data,unlabeled_data)

dic = {} 
i = 0
for index in nodes_index:
    dic[index] = clusterAssment[i]
    i+=1
dic = sorted(dic.items(), key=lambda d: d[0])
clusterAssment = np.array([ key_value[1] for key_value in dic])

dist = [{} for j in range(num_class)]
for i in range(num_node):
    clus_i = int(clusterAssment[i]) 
    centroids_j = centroids[clus_i] 
    dist_ij = distEclud(features[i],centroids_j) 
    dist[clus_i][i] = dist_ij 

In [9]:
normalized_similarities = {}
for i in range(num_class):
    normalized_similarity = get_normalized_similarity(dist[i])
    normalized_similarities.update(normalized_similarity)

In [10]:
features = torch.FloatTensor(features).cuda()
features_GCN = torch.FloatTensor(features_GCN).cuda()
labels = labels.cuda()
idx_train = idx_train.cuda()
idx_val = idx_val.cuda()
idx_test = idx_test.cuda()
labels_npy = labels.cpu().numpy()

features_npy = features.cpu().numpy()

In [11]:
best_acc = 0
num_epochs = batch_epochs[0]
model1 = MLP(nfeat=features.shape[1],
        nhid = hidden_size,
        nclass=labels.max().item() + 1,
        dropout=proxy_dp)
model1.cuda()
optimizer = optim.Adam(model1.parameters(),lr=0.2, weight_decay=5e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

t_total = time.time()
record = {}
for epoch in range(num_epochs):
    train(epoch,model1,record)
    scheduler.step()

        
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

############################################################################
current val_acc: 0.232
current test_acc: 0.181
############################################################################
current val_acc: 0.28400000000000003
current test_acc: 0.23700000000000002
############################################################################
current val_acc: 0.422
current test_acc: 0.363
############################################################################
current val_acc: 0.434
current test_acc: 0.437
############################################################################
current val_acc: 0.436
current test_acc: 0.425
############################################################################
current val_acc: 0.458
current test_acc: 0.396
############################################################################
current val_acc: 0.466
current test_acc: 0.40900000000000003
############################################################################
current val_a

In [12]:
model1 = model_best
output = model1(features)

loss_val1 = F.nll_loss(output[idx_val], labels[idx_val])
acc_val1 = accuracy(output[idx_val], labels[idx_val])

loss_test1 = F.nll_loss(output[idx_test], labels[idx_test])
acc_test1 = accuracy(output[idx_test], labels[idx_test])
print('Epoch: {:04d}'.format(epoch+1),
      'acc_val: {:.4f}'.format(acc_val1.item()),
      'acc_test: {:.4f}'.format(acc_test1.item()))

weight1 = 0.5*np.log(acc_val1.cpu().numpy() /(1-acc_val1.cpu().numpy() ))
output1 = F.softmax(model1(features),1).cpu().detach().numpy()
ensemble_output = output1
normalized_differences = np.zeros(num_node) 

predicted_class = np.argmax(ensemble_output,1)
pd.Series(predicted_class).value_counts().sort_index()

alpha = alphas[0]
weighted_score = {}
cluster_index = []
for i in range(num_class):
    cluster_index.append({})

for node in range(num_node):
    node_rep = normalized_similarities[node]
    node_inf = normalized_differences[node]
    weighted_score[node] = alpha*node_rep+(1-alpha)*node_inf
    
for node in idx_avaliable:
    cluster = predicted_class[node] 
    node_rep = normalized_similarities[node]
    node_inf = normalized_differences[node]
    cluster_index[cluster].update({node:weighted_score[node]})

for i in range(num_class):
    cluster_index[i] = sorted(cluster_index[i].items(), key=lambda d: d[1], reverse=True)

Epoch: 0121 acc_val: 0.4740 acc_test: 0.4090


In [13]:
batch_size = batch_sizes[1]
num_labeled_nodes_per_class = pd.Series(labels[idx_train].cpu().numpy()).value_counts().sort_index().values
num_labeled_nodes_per_class = len(idx_train)/num_class-num_labeled_nodes_per_class+batch_size
num_labeled_nodes_per_class = num_labeled_nodes_per_class.astype(int)
cut_zero(num_labeled_nodes_per_class)

In [14]:
class_pd = pd.Series(predicted_class[idx_avaliable]).value_counts().sort_index()
index = class_pd.index
values = class_pd.values
nodes_per_class = np.zeros(num_class)
for i in range(len(index)):
    nodes_per_class[index[i]] = values[i]
average_ab(num_labeled_nodes_per_class,nodes_per_class)

In [15]:
t = time.time()
idx_train = list(idx_train.cpu().numpy())
idx_avaliable_temp = copy.deepcopy(idx_avaliable)
count = np.zeros(num_class) 
while True:
    max_reception_node = get_max_receptive_node_dense(idx_train,idx_avaliable_temp,weighted_score)
    node_class = predicted_class[max_reception_node]
    if count[node_class] < num_labeled_nodes_per_class[node_class]:
        idx_train.append(max_reception_node) 
        idx_avaliable.remove(max_reception_node)
        idx_avaliable_temp.remove(max_reception_node)
        count[node_class] +=1
    else:
        idx_avaliable_temp.remove(max_reception_node)
        node_class = predicted_class[max_reception_node]
        for node in idx_avaliable_temp:
            if predicted_class[node]==node_class:
                idx_avaliable_temp.remove(node)       
    if np.sum(count) >=np.sum(num_labeled_nodes_per_class):
        break
print('node selection finished')

node selection finished


In [16]:
weight1 = 0.5*np.log(acc_val1.cpu().numpy() /(1-acc_val1.cpu().numpy() ))
output1 = F.softmax(model1(features),1).cpu().detach().numpy()
weights = [weight1]
outputs = [output1]

In [17]:
test_accs = []

In [18]:
for iteration in range(1,T-1):
    print('iteration:', iteration+1)
    initial_node_index = copy.deepcopy(idx_train)
    nodes_index = []
    for i in range(num_node): 
        if i not in initial_node_index:
            nodes_index.append(i)
    nodes_index = initial_node_index+nodes_index

    idx_train = torch.LongTensor(nodes_index[:len(initial_node_index)])
    idx_unlabeled = nodes_index[len(initial_node_index):] 
    train_label = np.array(labels_npy[idx_train]).reshape(-1,1)
    train_embeddings = np.array(features_npy[idx_train])


    labeled_data = np.hstack((train_embeddings,train_label))
    unlabeled_data =  np.array(features_npy[idx_unlabeled])
    clusterAssment,centroids = semi_kMeans(labeled_data,unlabeled_data)

    dic = {} 
    i = 0
    for index in nodes_index:
        dic[index] = clusterAssment[i]
        i+=1
    dic = sorted(dic.items(), key=lambda d: d[0])
    clusterAssment = np.array([ key_value[1] for key_value in dic])

    dist = [{} for j in range(num_class)]
    for i in range(num_node):
        clus_i = int(clusterAssment[i]) 
        centroids_j = centroids[clus_i] 
        dist_ij = distEclud(features_npy[i],centroids_j) 
        dist[clus_i][i] = dist_ij 

    normalized_similarities = {}
    for i in range(num_class):
        normalized_similarity = get_normalized_similarity(dist[i])
        normalized_similarities.update(normalized_similarity)
        
    idx_train = idx_train.cuda()
    best_acc = 0
    num_epochs = batch_epochs[iteration]
    model = copy.deepcopy(model_best)
    model.cuda()
    optimizer = optim.Adam(model.parameters(),lr=0.2, weight_decay=5e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    t_total = time.time()
    record = {}
    for epoch in range(num_epochs):
        train(epoch,model,record)
        scheduler.step()

    model = model_best
    output = model(features)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])

    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print('Epoch: {:04d}'.format(epoch+1),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'acc_test: {:.4f}'.format(acc_test.item()))
    
    
    weight = 0.5*np.log(acc_val.cpu().numpy() /(1-acc_val.cpu().numpy() ))
    weights.append(weight)
    output = F.softmax(model(features),1).cpu().detach().numpy()
    outputs.append(output)
    
    sum_weighted_dist = np.zeros(num_node)
    for i in range(iteration+1):
        for j in range(i+1,iteration+1):
            weighted_dist = get_weighted_dist(outputs[i],outputs[j],alphas[i],alphas[j])
            sum_weighted_dist+=weighted_dist
    ensemble_output = np.zeros((num_node,num_class))
    for i in range(len(weights)):
        ensemble_output +=weights[i]*outputs[i]
    predicted_class = np.argmax(ensemble_output,1)

    dist = [{} for _ in range(num_class) ]
    for i in range(num_node):
        clus_i = int(predicted_class [i]) 
        dist[clus_i][i] = sum_weighted_dist[i] 
    normalized_differences = {}
    for i in range(num_class):
        normalized_difference = get_normalized_difference(dist[i])
        normalized_differences.update(normalized_difference)

    
    alpha = alphas[iteration]
    weighted_score = {}
    for node in range(num_node):
        node_rep = normalized_similarities[node]
        node_inf = normalized_differences[node]
        weighted_score[node] = alpha*node_rep+(1-alpha)*node_inf


    batch_size = batch_sizes[iteration]
    num_labeled_nodes_per_class = pd.Series(labels[idx_train].cpu().numpy()).value_counts().sort_index().values
    num_labeled_nodes_per_class = len(idx_train)/num_class-num_labeled_nodes_per_class+batch_size
    num_labeled_nodes_per_class = num_labeled_nodes_per_class.astype(int)
    cut_zero(num_labeled_nodes_per_class)
    
    class_pd = pd.Series(predicted_class[idx_avaliable]).value_counts().sort_index()
    index = class_pd.index
    values = class_pd.values
    nodes_per_class = np.zeros(num_class)
    for i in range(len(index)):
        nodes_per_class[index[i]] = values[i]
    average_ab(num_labeled_nodes_per_class,nodes_per_class)

    print('@@@@@@@@@@@@@@@@@@@@@@@@@  num_labeled_nodes_per_class @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    print(num_labeled_nodes_per_class)
    
    idx_train = list(idx_train.cpu().numpy())
    idx_avaliable_temp = copy.deepcopy(idx_avaliable)
    count = np.zeros(num_class) 
    while True:
        max_reception_node = get_max_receptive_node_dense(idx_train,idx_avaliable_temp,weighted_score)
        
        if max_reception_node not in idx_avaliable_temp:
            print(pd.Series(predicted_class[idx_avaliable]).value_counts().sort_index())
        node_class = predicted_class[max_reception_node]
        if count[node_class] < num_labeled_nodes_per_class[node_class]:
            idx_train.append(max_reception_node) 
            idx_avaliable.remove(max_reception_node)
            if max_reception_node in idx_avaliable_temp:
                idx_avaliable_temp.remove(max_reception_node)
            count[node_class] +=1
        else:
            idx_avaliable_temp.remove(max_reception_node)
            node_class = predicted_class[max_reception_node]
            for node in idx_avaliable_temp:
                if predicted_class[node]==node_class:
                    idx_avaliable_temp.remove(node)       
        if np.sum(count) >=np.sum(num_labeled_nodes_per_class):
            break
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  Evaluation begin  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    best_acc2 = 0
    test_acc2 = 0
    model2 = GCN(nfeat=features_GCN.shape[1],
            nhid=hidden_size,
            nclass=labels.max().item() + 1,
            dropout=0.85)
    model2.cuda()
    optimizer = optim.Adam(model2.parameters(),
                           lr=0.05, weight_decay=5e-4)
    t_total = time.time()
    record = {}
    for epoch in range(400):
        train2(epoch,model2,record)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    bit_list = sorted(record.keys())
    bit_list.reverse()
    for key in bit_list[:10]:
        value = record[key]
        print(key,value)
    test_accs.append(test_acc2)
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  Evaluation end  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

iteration: 2
############################################################################
current val_acc: 0.29
current test_acc: 0.325
############################################################################
current val_acc: 0.358
current test_acc: 0.34900000000000003
############################################################################
current val_acc: 0.364
current test_acc: 0.327
############################################################################
current val_acc: 0.374
current test_acc: 0.36
############################################################################
current val_acc: 0.558
current test_acc: 0.542
############################################################################
current val_acc: 0.5640000000000001
current test_acc: 0.601
############################################################################
current val_acc: 0.5720000000000001
current test_acc: 0.522
Epoch: 0116 acc_val: 0.5720 acc_test: 0.5220
@@@@@@@@@@@@@@@@@@@@@@@@@  num_label

@@@@@@@@@@@@@@@@@@@@@@@@@  num_labeled_nodes_per_class @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
[1 2 1 0 0 2]
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  Evaluation begin  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
###########################update testacc#####################################
current val_acc: 0.232
current test_acc: 0.181
###########################update testacc#####################################
current val_acc: 0.31
current test_acc: 0.281
###########################update testacc#####################################
current val_acc: 0.384
current test_acc: 0.365
###########################update testacc#####################################
current val_acc: 0.454
current test_acc: 0.396
###########################update testacc#####################################
current val_acc: 0.458
current test_acc: 0.392
###########################update testacc#####################################
current val_acc: 0.496
current test_acc: 0.507
###########################update testacc#

############################################################################
current val_acc: 0.178
current test_acc: 0.213
############################################################################
current val_acc: 0.45
current test_acc: 0.452
############################################################################
current val_acc: 0.5760000000000001
current test_acc: 0.583
############################################################################
current val_acc: 0.646
current test_acc: 0.629
############################################################################
current val_acc: 0.658
current test_acc: 0.658
############################################################################
current val_acc: 0.666
current test_acc: 0.674
############################################################################
current val_acc: 0.67
current test_acc: 0.685
############################################################################
current val_acc: 0.672
current test_acc: 0.7

Epoch: 0083 acc_val: 0.7000 acc_test: 0.7030
@@@@@@@@@@@@@@@@@@@@@@@@@  num_labeled_nodes_per_class @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
[1 2 1 0 1 1]
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  Evaluation begin  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
###########################update testacc#####################################
current val_acc: 0.198
current test_acc: 0.187
###########################update testacc#####################################
current val_acc: 0.302
current test_acc: 0.255
###########################update testacc#####################################
current val_acc: 0.39
current test_acc: 0.398
###########################update testacc#####################################
current val_acc: 0.58
current test_acc: 0.587
###########################update testacc#####################################
current val_acc: 0.624
current test_acc: 0.668
###########################update testacc#####################################
current val_acc: 0.632
current test_acc: 0.65

############################################################################
current val_acc: 0.212
current test_acc: 0.231
############################################################################
current val_acc: 0.462
current test_acc: 0.439
############################################################################
current val_acc: 0.55
current test_acc: 0.548
############################################################################
current val_acc: 0.58
current test_acc: 0.5630000000000001
############################################################################
current val_acc: 0.586
current test_acc: 0.599
############################################################################
current val_acc: 0.71
current test_acc: 0.6980000000000001
############################################################################
current val_acc: 0.73
current test_acc: 0.707
Epoch: 0069 acc_val: 0.7300 acc_test: 0.7070
@@@@@@@@@@@@@@@@@@@@@@@@@  num_labeled_nodes_per_class @@@@@@@@@@

Optimization Finished!
Total time elapsed: 3.4131s
0.72 0.732
0.716 0.722
0.714 0.724
0.712 0.706
0.71 0.714
0.708 0.728
0.706 0.714
0.704 0.71
0.7020000000000001 0.719
0.7000000000000001 0.714
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  Evaluation end  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
iteration: 16
############################################################################
current val_acc: 0.316
current test_acc: 0.342
############################################################################
current val_acc: 0.452
current test_acc: 0.434
############################################################################
current val_acc: 0.526
current test_acc: 0.515
############################################################################
current val_acc: 0.588
current test_acc: 0.636
############################################################################
current val_acc: 0.664
current test_acc: 0.684
########################################################################

###########################update testacc#####################################
current val_acc: 0.734
current test_acc: 0.724
Optimization Finished!
Total time elapsed: 3.5295s
0.734 0.724
0.732 0.734
0.726 0.727
0.724 0.723
0.722 0.729
0.72 0.726
0.718 0.716
0.716 0.736
0.714 0.727
0.712 0.718
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  Evaluation end  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


In [19]:
print(pd.Series(labels_npy[idx_train]).value_counts().sort_index())
print(len(idx_train)/num_class)
print((1815-len(idx_avaliable))/num_class)
reception_fileds = get_current_neighbors_dense(idx_train).sum()
print(reception_fileds)

0    19
1    22
2    21
3    19
4    20
5    19
dtype: int64
20.0
18.0
1719


#### Model evaluation

In [20]:
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid,bias=True)
        self.gc2 = GraphConvolution(nhid, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return x
    
best_acc2 = 0
test_acc2 = 0
def train2(epoch, model,record):
    global best_acc2
    global test_acc2
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features_GCN, adj)
    loss_train = F.cross_entropy(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    model.eval()
    output = model(features_GCN, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])

    record[acc_val.item()] = acc_test.item()
    
    if acc_val > best_acc2:
        print("###########################update testacc#####################################")
        print("current val_acc:", acc_val.item())
        print("current test_acc:", acc_test.item())
        best_acc2 = acc_val
        test_acc2 = acc_test.item()

In [21]:
best_acc2 = 0
test_acc2 = 0
model2 = GCN(nfeat=features_GCN.shape[1],
        nhid=hidden_size,
        nclass=labels.max().item() + 1,
        dropout=0.85)
model2.cuda()
optimizer = optim.Adam(model2.parameters(),
                       lr=0.05, weight_decay=5e-4)
t_total = time.time()
record = {}
for epoch in range(400):
    train2(epoch,model2,record)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

###########################update testacc#####################################
current val_acc: 0.352
current test_acc: 0.28600000000000003
###########################update testacc#####################################
current val_acc: 0.494
current test_acc: 0.44
###########################update testacc#####################################
current val_acc: 0.508
current test_acc: 0.507
###########################update testacc#####################################
current val_acc: 0.612
current test_acc: 0.605
###########################update testacc#####################################
current val_acc: 0.634
current test_acc: 0.607
###########################update testacc#####################################
current val_acc: 0.662
current test_acc: 0.63
###########################update testacc#####################################
current val_acc: 0.664
current test_acc: 0.653
###########################update testacc#####################################
current val_acc: 0.706
curr

In [22]:
test_accs

[0.651,
 0.668,
 0.651,
 0.666,
 0.705,
 0.6880000000000001,
 0.6970000000000001,
 0.715,
 0.7000000000000001,
 0.727,
 0.712,
 0.721,
 0.736,
 0.729,
 0.746,
 0.74,
 0.724]